<a href="https://colab.research.google.com/github/toddlack/tortoise-tts/blob/dev/tortoise_tts_colab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

## Colab specific setup
mount google drive, install python 3.12

In [ ]:
# prompt: mount google drive
from google.colab import drive
drive.mount('/content/drive')


Create new python environment called 'tortoise' and install pytorch

In [1]:
%%bash
# sudo apt-get update -y
# sudo apt-get install python3.12
PYTHONPATH=
pip install  condacolab

In [2]:

import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


In [3]:
%%bash
PYTHONPATH=
# conda update -n base -c conda-forge conda
conda install pytorch torchvision torchaudio pytorch-cuda -c pytorch -c nvidia

conda create --name tortoise python=3.10 inflect
source activate tortoise
conda info --envs
python --version


Channels:
 - pytorch
 - nvidia
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pytorch
    - pytorch-cuda
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    blas-2.116                 |              mkl          13 KB  conda-forge
    blas-devel-3.9.0           |   16_linux64_mkl          12 KB  conda-forge
    ca-certificates-2024.12.14 |       hbcca054_0         153 KB  conda-forge
    certifi-2024.12.14         |     pyhd8ed1ab_0         158 KB  conda-forge
    cuda-cudart-12.4.127       |                0         198 KB  nvidia
    cuda-cupti-12.4.127        |                0        16.4 MB  nvidia
    cuda-libraries-12.4.1      |                0           2 K



==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.11.2

Please update conda by running

    $ conda update -n base -c conda-forge conda




==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.11.2

Please update conda by running

    $ conda update -n base -c conda-forge conda




In [1]:
# @title Clone repo for local use
#Clone a repo and cd into the new directory
content_root = "/content" # @param {"type":"string","placeholder":"content root"}
git_repo='https://github.com/toddlack/tortoise-tts.git'  # @param {"type":"string","placeholder":"git repo url"}
git_branch='dev'  # @param {"type":"string","placeholder":"git branch"}
app_root=git_repo.split('/')[-1].replace('.git', '')
assets_dir=f'{content_root}/{app_root}/assets'
%cd {content_root}
!git clone --single-branch -b {git_branch} {git_repo}
%cd {app_root}


/content
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 2010, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 2010 (delta 11), reused 0 (delta 0), pack-reused 1990 (from 3)
Receiving objects: 100% (2010/2010), 54.66 MiB | 12.68 MiB/s, done.
Resolving deltas: 100% (922/922), done.
/content/tortoise-tts


In [9]:
# @markdown Run this cell to Update git repo
%cd {app_root}
!git pull

[Errno 2] No such file or directory: 'tortoise-tts'
/content/tortoise-tts
Already up to date.


In [7]:
# first follow the instructions in the README.md file under Local Installation
#%pip install -r requirements.txt
%%bash
#Set python environment before installing tortoise-tts
source activate tortoise
pip install numpy
pip install -e  .
conda info --envs
# conda install .




Obtaining file:///content/tortoise-tts
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: tortoise-tts
    Found existing installation: tortoise-tts 3.0.0
    Uninstalling tortoise-tts-3.0.0:
      Successfully uninstalled tortoise-tts-3.0.0
  Running setup.py develop for tortoise-tts
# conda environments:
#
base                     /usr/local
tortoise              *  /usr/local/envs/tortoise



  DEPRECATION: Legacy editable install of tortoise-tts==3.0.0 from file:///content/tortoise-tts (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


In [6]:
# @markdown Install latest tortoise-tts
!pip uninstall tortoise-tts
# !pip install git+https://github.com/neonbjb/tortoise-tts

## Main code

In [8]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
# tts = TextToSpeech()
# If you want to use deepspeed then pass use_deepspeed=True nearly 2x faster than normal
tts = TextToSpeech(use_deepspeed=True, kv_cache=True)

RuntimeError: Failed to import transformers.models.gpt2.modeling_gpt2 because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
module 'numpy._core' has no attribute 'multiarray'

In [19]:
# Reactivate the conda environment if needed:
!source activate tortoise
# Uninstall conflicting libraries with pip:
!pip uninstall -y transformers accelerate
# Update/reinstall NumPy:
!conda install -y numpy
# Reinstall transformers and accelerate with conda:
!conda install -c conda-forge transformers accelerate

Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: \ | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.11.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.11.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - accelerate
    - transformers


The following packages will be downloaded:

    package                    |            bu

In [ ]:
# @title text and preset

# This is the text that will be spoken.
text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?"

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "ultra_fast"

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create a list of text input widgets
num_texts = 3  # Number of text inputs in the array
text_inputs = [widgets.Text(placeholder=f'Text {i+1}') for i in range(num_texts)]

# Display the text inputs vertically
display(widgets.VBox(text_inputs))

# Function to retrieve the text values
def get_texts():
  return [text_input.value for text_input in text_inputs]

# Example usage:
texts = get_texts()
print(texts)

In [ ]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls tortoise/voices

IPython.display.Audio('tortoise/voices/tom/1.wav')

In [ ]:
# Pick one of the voices from the output above
voice = 'train_daws'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

## Other actions

In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.",
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

In [ ]:
del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.

# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=train_atkins --textfile=tortoise/data/riding_hood.txt --preset=ultra_fast --output_path=.

IPython.display.Audio('train_atkins/combined.wav')
# This will take awhile..